In [1]:
import pandas as pd
import mysql.connector

# connect to mysql
conn = mysql.connector.connect(user='root', password='password',
                              host='127.0.0.1', port=3306, database='store_transactions',
                              auth_plugin='mysql_native_password')
cur = conn.cursor()
    
# read csv with pandas
csv_data = pd.read_csv("transactions_csv.csv") 

In [2]:
# insert data into products table
product_df = csv_data['product_name'].str.lower().drop_duplicates()
for row in product_df:
    cur.execute('''INSERT IGNORE INTO products (name) VALUES ('%s')''' % row)
    conn.commit()

# insert data into stores table
store_df = csv_data['store_name'].str.title().drop_duplicates()
for row in store_df:
    cur.execute('''INSERT IGNORE INTO stores (name) VALUES ('%s')''' % row)
    conn.commit()
    
# insert data into sales_reps table
sales_rep_df = pd.DataFrame(csv_data.sales_representative_name.str.title().\
    drop_duplicates().str.split(' ',1).tolist(), columns = ['first_name','last_name'])
for i, row in sales_rep_df.iterrows():
    cur.execute('''INSERT IGNORE INTO sales_reps (first_name, last_name) VALUES \
    ("%s", "%s")''' % (row.first_name, row.last_name))
    conn.commit()

# insert data into customers table
customer_df = pd.DataFrame(csv_data.client_name.str.title().\
    drop_duplicates().str.split(' ',1).tolist(), columns = ['first_name','last_name'])
for i, row in customer_df.iterrows():
    cur.execute('''INSERT IGNORE INTO customers (first_name, last_name) VALUES \
    ("%s", "%s")''' % (row.first_name, row.last_name))
    conn.commit()

# insert data into transaction/order table
for index, row in csv_data.iterrows():
    # get FK ids
    sql = '''SELECT 
    (SELECT product_id FROM products WHERE name = lower('{0}')) as product_id, \
    (SELECT store_id FROM stores WHERE name = lower('{1}')) as store_id, \
    (SELECT customer_id FROM customers WHERE first_name  = SUBSTRING_INDEX('{2}', ' ', 1) \
        AND  last_name = SUBSTRING_INDEX('{2}', ' ', -1)) as customer_id, \
    (SELECT sales_rep_id FROM sales_reps WHERE first_name = SUBSTRING_INDEX('{3}', ' ', 1) \
        AND  last_name = SUBSTRING_INDEX('{3}', ' ', -1)) as sales_rep_id '''.\
    format(row.product_name, row.store_name, row.client_name.title(), \
           row.sales_representative_name.title())
    cur.execute(sql)
    prod_id, store_id, cust_id, rep_id = cur.fetchone()

    transaction_df = (row.transaction_id, row.transaction_date, prod_id, store_id, cust_id, rep_id, row.price)
    cur.execute('''INSERT IGNORE INTO transactions(transaction_id, transaction_date, product_id, \
               store_id, customer_id, sales_rep_id, price) VALUES (%s, %s, %s, %s, %s, %s, %s)''', transaction_df)
    conn.commit()

In [3]:
#execute query
sql = '''
select c.first_name, c.last_name, (select sum(price) from transactions where customer_id = c.customer_id) as total_purchases
from transactions tr
join stores s using(store_id)
join products pr using(product_id)
join customers c using(customer_id)
join (SELECT customer_id, store_id, max(transaction_date) as last_date FROM transactions group by customer_id, store_id) tr2 on tr.customer_id=tr2.customer_id and tr.store_id = tr2.store_id
and tr.transaction_date = last_date
where s.name = '%s'
and pr.name = '%s'
order by total_purchases desc''' % ('King St', 'cappuccino')
cur.execute(sql)
myresult = cur.fetchall()
pd.DataFrame(myresult, columns = ["first_name", "last_name", "total_purchase"])

,first_name,last_name,total_purchase
0,Ananya,Werner,12.4
1,Miranda,Nash,8.5
2,Gabriel,Palmer,4.5


In [5]:
# Close connection
conn.close


<bound method MySQLConnection.close of <mysql.connector.connection.MySQLConnection object at 0x000002241F1C3F88>>